In [3]:
%env METIS_DLL=/home/juan/Descargas/metis-5.0/build/Linux-x86_64/libmetis/libmetis.so

env: METIS_DLL=/home/juan/Descargas/metis-5.0/build/Linux-x86_64/libmetis/libmetis.so


In [1]:
import scipy

scipy.version.version

'1.5.2'

In [2]:
from scipy.sparse import lil_matrix, csr_matrix
import numpy as np
#import metis
from clustering_evaluation import computeEvaluationMeasures, MEASURES
import glob
from tabulate import tabulate
from tqdm import tqdm
import os

def _loadSparseCSRFromDisk(fullPath):
    spD = None
    with open(fullPath, 'rb') as f:
        data = np.load(f)
        indices = np.load(f)
        indptr = np.load(f)
        spD = csr_matrix((data, indices, indptr))
        # convert this nxk matrix into a similarity.
    return spD

def _SaveSparseCSRToDisk(denseFn, spM):
    """
    denseFn: output file 
    spM: sparse matrix to convert and store.
    """
    csrOut = "{0}.npy".format(denseFn[: -(len(denseFn.split('.')[-1])+1)]) # replaces the file extension.
    print(csrOut)
    #M  = np.loadtxt(denseFn, delimiter=',')
    #spM = csr_matrix(M)
    with open(csrOut, 'wb') as f:
        np.save(f, spM.data)
        np.save(f, spM.indices)
        np.save(f, spM.indptr)
    """
    When loading
    >> with open('test.npy', 'rb') as f:
    >>     a = np.load(f)
    >>     b = np.load(f)
    >> csr_matrix((data, indices, indptr))
    """

def getLabelsFromFile(fullPath):
    v = np.loadtxt(fullPath, delimiter=',', dtype=np.int)
    return v

def detectPostfixesInDir(fullPath, dirName):
    fnLst = glob.glob('{0}/{1}/*labels_*.out'.format(fullPath, dirName))
    # labels_NCF_DS_
    postFixes = [f.split(os.sep)[-1].replace('labels_NCF_DS_','').replace('.out','') for f in fnLst]
    return postFixes

#path = '/home/juan/Insync/juan.zamora@pucv.cl/Google Drive/Research - Multiview and Collaborative Clustering/code/handwritten_tezt/similarity_matrices_after_merging'

In [3]:
from scipy.sparse import csc_matrix,csr_matrix, lil_matrix
from itertools import product
#from spectral_embedding import *
from sklearn.cluster import MiniBatchKMeans
from sklearn.manifold import SpectralEmbedding, spectral_embedding
from clustering_evaluation import computeEvaluationMeasures, MEASURES
from sklearn.metrics.pairwise import pairwise_kernels


def getSimClus(D):
    """
    Computes a matrix that contains the intersection of items
    D is a csc_matrix with n rows denoting data points and k columns denoting clusters.
    D_ij denotes the membership score of data point i in cluster j.
    """
    D_asCol = csc_matrix(D) # more efficient for making column slices

    # create a kxk matrix with zeros
    k = D.shape[1]
    simClus = np.zeros((k,k))
    # compare each pair of clusters from D
    for i in range(k):
        sClust_i = set(D_asCol[:,i].nonzero()[0])
        for j in range(i+1,k):
            sClust_j = set(D_asCol[:,j].nonzero()[0])
            simClus[i,j] = len(sClust_i & sClust_j) / len(sClust_i | sClust_j)
            simClus[j,i] = simClus[i,j]
    return simClus


def compute_aSRS(D, DC=1.0):
    # aim: compute the approximate SimRankSimilarity matrix (aSRS).
    # compute for each pair of data points i,j the total sum of simclus values 
    n = D.shape[0]
    
    simClus = getSimClus(D)

    aSRS = np.zeros((n,n))
    for i in range(n):
        nbors_i = D[i].nonzero()[1]
        n_nbors_i = len(nbors_i)
        for j in range(i+1, n):
            nbors_j = D[j].nonzero()[1]
            n_nbors_j = len(nbors_j)
            aSRS[i,j] = (DC/(n_nbors_i*n_nbors_j))*np.sum([simClus[ca,cb] for ca, cb in list(product(nbors_i, nbors_j))])
            aSRS[j,i] = aSRS[i,j]
    return aSRS

# old method, use it only as an example. Below it is the correct version!
def _deprecated_perform_evaluate_runs(dsDirPath, nruns=10, threshold=0.0, embDim=4, kernelFn='rbf'):
    trueLblsFname = "{0}/{1}/{2}".format(path, dsDirPath, 'labels.true')
    trueL = getLabelsFromFile(trueLblsFname)
    postFixesDS = detectPostfixesInDir(path, dsDirPath)

    dsresults = []
    header = ['seed']
    header.extend(MEASURES)
    np.random.seed(87009)
    n_clusters = np.unique(trueL).shape[0]
    runSeed = np.random.randint(0, high=2e8)
    for runPostfix in tqdm(postFixesDS):#tqdm(postFixesDS):
        #print("{0}/{1}{2}_csr.npy".format(dsDirPath, 'simmatrix_DS_', runPostfix))        
        simFname = "{0}/{1}/{2}{3}_csr.npy".format(path, dsDirPath, 'simmatrix_DS_', runPostfix)
        D = _loadSparseCSRFromDisk(simFname)

        #aSRS = compute_aSRS(D)
        #outSRSFn = "{0}/{1}/{2}{3}.npy".format(path, dsDirPath, 'aSRS_DS_', runPostfix)
        #np.save(outSRSFn, aSRS)
        ##aSRS = np.load(outSRSFn)
        
        # builds the embedded vectors
        embedding = spectral_embedding(pairwise_kernels(D, metric=kernelFn), n_components=embDim, norm_laplacian=True, drop_first=True)
        
        #spEmbMod = SpectralEmbedding(n_components=20, affinity='nearest_neighbors', n_jobs=4, eigen_solver='arpack', random_state=runSeed)        
        #embedding = spEmbMod.fit_transform(D.toarray())
        
        #outEmbFn = "{0}/{1}/{2}{3}.npy".format(path, dsDirPath, 'embedding_DS_', runPostfix)
        #np.save(outEmbFn, embedding)
        #embedding = np.load(outEmbFn)
        
        # old, not working!
        #spectralEmb = SpectralEmbedding(embedding_dimension = 30, node_weights = 'degree')
        #spectralEmb.fit(aSRS)
        #embedding = spectralEmb.embedding_
        
        runDsResults = [] # a single run passes through all the npy matrices with the same seed        
        for run in tqdm(range(nruns)): # the partitioning method has a stochastic part, hence several runs are needed.
            kmeans = MiniBatchKMeans(n_clusters, random_state=np.random.randint(0, high=2e8))
            kmeans.fit(embedding)
            pred_labels = list(kmeans.labels_)

            perf = computeEvaluationMeasures(trueL, pred_labels)
            runDsResults.append([perf[m] for m in MEASURES])

        runRow = [runSeed]
        runRow.extend(list(np.mean(runDsResults, axis=0)))
        dsresults.append( runRow ) # average

    dsresults.append( ['Average'] + list(np.mean(dsresults, axis=0)[1:]) )
    print(tabulate(dsresults, headers=header, tablefmt='github', floatfmt='.6f', showindex=True))    

In [4]:
def embed_and_consensus(dsDirPath, nruns=10, threshold=0.0, embDim=4, kernelFn='rbf', precomputed_sim=False):
    """
    precomputed_sim: Signals if the file with the soft-clustering based similarity matrix is already computed and stored.
    """
    trueLblsFname = "{0}/{1}/{2}".format(path, dsDirPath, 'labels.true')
    trueL = getLabelsFromFile(trueLblsFname)
    postFixesDS = detectPostfixesInDir(path, dsDirPath)

    dsresults = []
    header = ['seed']
    header.extend(MEASURES)
    np.random.seed(87009)
    n_clusters = np.unique(trueL).shape[0]
    runSeed = np.random.randint(0, high=2e8)
    
    t = tqdm(postFixesDS, leave=False)
    
    for runPostfix in t:

        simFname = "{0}/{1}/{2}{3}_csr.npy".format(path, dsDirPath, 'dotmatrix_DS_', runPostfix)
        
        if not precomputed_sim:
            softClusFN = "{0}/{1}/{2}{3}.out".format(path, dsDirPath, 'simmatrix_DS_', runPostfix)
            print("Opening soft-clustering matrix from {0}".format(softClusFN))
            scM = csr_matrix(np.loadtxt(softClusFN, delimiter=',') )# soft-clustering matrix        
            #scM = np.loadtxt(softClusFN, delimiter=',') # doesn't affect because dot-sim doesn't change!
            #scM = csr_matrix(scM[:,~np.all(scM == 0, axis = 0)])

                        
            # similarity computation (numpy pairwise... for dense matrices and .dot for sparse.)
            #simM = pairwise_kernels(scM, metric=kernelFn) # dot prod between soft-clus matrix and itself (transp.)
            simM = scM.dot(scM.T)
            
            print("Storing similarity matrix in sparse format to {0}".format(simFname))
            _SaveSparseCSRToDisk("{0}/{1}/{2}{3}_csr.npy".format(path, dsDirPath, 'dotmatrix_DS_', runPostfix), simM)
        else:
            t.set_description("Opening soft-clustering based similarity matrix from {0}".format(simFname))
            #print("Opening soft-clustering based similarity matrix from {0}".format(simFname))
            simM = _loadSparseCSRFromDisk(simFname)
        
        embedding = spectral_embedding(simM, n_components=embDim, norm_laplacian=True, drop_first=True)        
        
        runDsResults = [] # a single run passes through all the npy matrices with the same seed        
        t2 = tqdm(range(nruns), leave=False)
        for run in t2: # the partitioning method has a stochastic part, hence several runs are needed.
            t2.set_description("run {0}/{1}".format(run, nruns))
            kmeans = MiniBatchKMeans(n_clusters, random_state=np.random.randint(0, high=2e8))
            kmeans.fit(embedding)
            pred_labels = list(kmeans.labels_)

            perf = computeEvaluationMeasures(trueL, pred_labels)
            runDsResults.append([perf[m] for m in MEASURES])

        runRow = [runSeed]
        runRow.extend(list(np.mean(runDsResults, axis=0)))
        dsresults.append( runRow ) # average

    dsresults.append( ['Average'] + list(np.mean(dsresults, axis=0)[1:]) )
    print(tabulate(dsresults, headers=header, tablefmt='github', floatfmt='.6f', showindex=True))

# Summary of benchmark results

In [38]:
from scipy.io import loadmat
import os

resultPath = '../matlab-experiments/results/'
runDsResults = []

fnLst = glob.glob('{0}/*_ensemble_*.mat'.format(resultPath))
for fname in map(os.path.basename, fnLst):
    res = loadmat('{0}/{1}'.format(resultPath, fname))#'cspa', 'hgpa', 'mcla', 'trLbls'
    for method in ('cspa', 'hgpa', 'mcla'):
        perf = computeEvaluationMeasures(res['trLbls'].flatten(), res[method].flatten())
        runDsResults.append([fname, method]+[perf[m] for m in MEASURES])
        
fnLst = glob.glob('{0}/*_negMM_*.mat'.format(resultPath))
for fname in map(os.path.basename, fnLst):
    res = loadmat('{0}/{1}'.format(resultPath, fname))#'negMMLabels'
    perf = computeEvaluationMeasures(res['trLbls'].flatten(), res['negMMLabels'].flatten())
    runDsResults.append([fname, 'negMM']+[perf[m] for m in MEASURES])

header = ['fname', 'method']
header.extend(MEASURES)    
print(tabulate(runDsResults, headers=header, tablefmt='github', floatfmt='.4f', showindex=True))

c:\users\juan\.virtualenvs\multiview-clustering-ahpdxa8l\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\juan\.virtualenvs\multiview-clustering-ahpdxa8l\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\juan\.virtualenvs\multiview-clustering-ahpdxa8l\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\use

|    | fname                            | method   |      E |      P |     F1 |    ACC |    NMI |   PREC |    REC |     ARI |
|----|----------------------------------|----------|--------|--------|--------|--------|--------|--------|--------|---------|
|  0 | bbc2_ensemble_results.mat        | cspa     | 0.3472 | 0.8226 | 0.1055 | 0.0820 | 0.5350 | 0.1478 | 0.0820 |  0.3889 |
|  1 | bbc2_ensemble_results.mat        | hgpa     | 0.3957 | 0.7639 | 0.0678 | 0.0562 | 0.4968 | 0.0868 | 0.0562 |  0.3698 |
|  2 | bbc2_ensemble_results.mat        | mcla     | 0.3481 | 0.8221 | 0.0099 | 0.0080 | 0.5404 | 0.0140 | 0.0080 |  0.4352 |
|  3 | bbc3_ensemble_results.mat        | cspa     | 0.3430 | 0.8226 | 0.0173 | 0.0142 | 0.5290 | 0.0227 | 0.0142 |  0.3825 |
|  4 | bbc3_ensemble_results.mat        | hgpa     | 0.5489 | 0.6467 | 0.2348 | 0.1924 | 0.3603 | 0.3524 | 0.1924 |  0.2601 |
|  5 | bbc3_ensemble_results.mat        | mcla     | 0.3995 | 0.7989 | 0.2222 | 0.1814 | 0.4852 | 0.3501 | 0.1814 |  0

c:\users\juan\.virtualenvs\multiview-clustering-ahpdxa8l\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\juan\.virtualenvs\multiview-clustering-ahpdxa8l\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\users\juan\.virtualenvs\multiview-clustering-ahpdxa8l\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\use

|    | fname                            | method   |      E |      P |     F1 |    ACC |    NMI |   PREC |    REC |     ARI |
|----|----------------------------------|----------|--------|--------|--------|--------|--------|--------|--------|---------|
|  0 | bbc2_ensemble_results.mat        | cspa     | 0.3472 | 0.8226 | 0.1055 | 0.0820 | 0.5350 | 0.1478 | 0.0820 |  0.3889 |
|  1 | bbc2_ensemble_results.mat        | hgpa     | 0.3957 | 0.7639 | 0.0678 | 0.0562 | 0.4968 | 0.0868 | 0.0562 |  0.3698 |
|  2 | bbc2_ensemble_results.mat        | mcla     | 0.3481 | 0.8221 | 0.0099 | 0.0080 | 0.5404 | 0.0140 | 0.0080 |  0.4352 |
|  3 | bbc3_ensemble_results.mat        | cspa     | 0.3430 | 0.8226 | 0.0173 | 0.0142 | 0.5290 | 0.0227 | 0.0142 |  0.3825 |
|  4 | bbc3_ensemble_results.mat        | hgpa     | 0.5489 | 0.6467 | 0.2348 | 0.1924 | 0.3603 | 0.3524 | 0.1924 |  0.2601 |
|  5 | bbc3_ensemble_results.mat        | mcla     | 0.3995 | 0.7989 | 0.2222 | 0.1814 | 0.4852 | 0.3501 | 0.1814 |  0.3789 |
|  6 | bbc4_ensemble_results.mat        | cspa     | 0.4327 | 0.7182 | 0.0418 | 0.0277 | 0.4308 | 0.0878 | 0.0277 |  0.2698 |
|  7 | bbc4_ensemble_results.mat        | hgpa     | 0.4689 | 0.6978 | 0.0150 | 0.0117 | 0.4024 | 0.0280 | 0.0117 |  0.2732 |
|  8 | bbc4_ensemble_results.mat        | mcla     | 0.4643 | 0.7518 | 0.1228 | 0.1066 | 0.4100 | 0.1448 | 0.1066 |  0.2626 |
|  9 | bbcsp2_ensemble_results.mat      | cspa     | 0.3897 | 0.7463 | 0.1618 | 0.1305 | 0.4647 | 0.2205 | 0.1305 |  0.2769 |
| 10 | bbcsp2_ensemble_results.mat      | hgpa     | 0.4729 | 0.6912 | 0.1160 | 0.0956 | 0.3954 | 0.1494 | 0.0956 |  0.2288 |
| 11 | bbcsp2_ensemble_results.mat      | mcla     | 0.9406 | 0.3548 | 0.0233 | 0.1140 | 0.0000 | 0.0130 | 0.1140 |  0.0000 |
| 12 | bbcsp3_ensemble_results.mat      | cspa     | 0.4983 | 0.6560 | 0.0987 | 0.0709 | 0.3434 | 0.1997 | 0.0709 |  0.1720 |
| 13 | bbcsp3_ensemble_results.mat      | hgpa     | 0.5507 | 0.6241 | 0.1799 | 0.1277 | 0.3000 | 0.3202 | 0.1277 |  0.1615 |
| 14 | bbcsp3_ensemble_results.mat      | mcla     | 0.5247 | 0.6489 | 0.1766 | 0.1241 | 0.3327 | 0.3592 | 0.1241 |  0.1710 |
| 15 | bbcsp4_ensemble_results.mat      | cspa     | 0.7382 | 0.4569 | 0.1497 | 0.1121 | 0.1458 | 0.2772 | 0.1121 |  0.0282 |
| 16 | bbcsp4_ensemble_results.mat      | hgpa     | 0.7915 | 0.4052 | 0.0897 | 0.0603 | 0.1004 | 0.1820 | 0.0603 |  0.0042 |
| 17 | bbcsp4_ensemble_results.mat      | mcla     | 0.7284 | 0.4741 | 0.1152 | 0.0948 | 0.1608 | 0.1729 | 0.0948 |  0.0533 |
| 18 | caltech-20_ensemble_results.mat  | cspa     | 0.4003 | 0.6639 | 0.0027 | 0.0092 | 0.4960 | 0.0016 | 0.0092 |  0.3176 |
| 19 | caltech-20_ensemble_results.mat  | hgpa     | 0.4415 | 0.5985 | 0.0871 | 0.0650 | 0.4450 | 0.1545 | 0.0650 |  0.2684 |
| 20 | caltech-20_ensemble_results.mat  | mcla     | 0.3918 | 0.6601 | 0.0655 | 0.0813 | 0.5118 | 0.0564 | 0.0813 |  0.3843 |
| 21 | handwritten_ensemble_results.mat | cspa     | 0.2040 | 0.8890 | 0.1000 | 0.1000 | 0.7960 | 0.1000 | 0.1000 |  0.7759 |
| 22 | handwritten_ensemble_results.mat | hgpa     | 0.3215 | 0.7570 | 0.2057 | 0.2080 | 0.6807 | 0.2068 | 0.2080 |  0.5991 |
| 23 | handwritten_ensemble_results.mat | mcla     | 0.2811 | 0.7885 | 0.1206 | 0.1190 | 0.7196 | 0.1224 | 0.1190 |  0.6500 |
| 24 | nusWide_ensemble_results.mat     | cspa     | 0.8186 | 0.2008 | 0.0067 | 0.0144 | 0.0958 | 0.0047 | 0.0144 |  0.0456 |
| 25 | nusWide_ensemble_results.mat     | hgpa     | 0.8926 | 0.1196 | 0.0094 | 0.0205 | 0.0012 | 0.0065 | 0.0205 | -0.0001 |
| 26 | nusWide_ensemble_results.mat     | mcla     | 0.8072 | 0.2247 | 0.0055 | 0.0137 | 0.1112 | 0.0038 | 0.0137 |  0.0643 |
| 27 | yahoo_ensemble_results.mat       | cspa     | 0.1959 | 0.8842 | 0.0820 | 0.0675 | 0.4116 | 0.2104 | 0.0675 |  0.0980 |
| 28 | yahoo_ensemble_results.mat       | hgpa     | 0.5042 | 0.7038 | 0.0695 | 0.0389 | 0.1503 | 0.3899 | 0.0389 |  0.0341 |
| 29 | yahoo_ensemble_results.mat       | mcla     | 0.3311 | 0.8051 | 0.0622 | 0.0457 | 0.3151 | 0.1970 | 0.0457 |  0.1009 |
| 30 | bbc2_negMM_results.mat           | negMM    | 0.1389 | 0.9558 | 0.3770 | 0.3757 | 0.8606 | 0.3783 | 0.3757 |  0.8919 |
| 31 | bbc3_negMM_results.mat           | negMM    | 0.1480 | 0.9535 | 0.0042 | 0.0047 | 0.8502 | 0.0038 | 0.0047 |  0.8846 |
| 32 | bbc4_negMM_results.mat           | negMM    | 0.3251 | 0.7971 | 0.4417 | 0.4248 | 0.6458 | 0.6092 | 0.4248 |  0.5783 |
| 33 | bbcsp2_negMM_results.mat         | negMM    | 0.1435 | 0.9467 | 0.1872 | 0.1857 | 0.8480 | 0.1892 | 0.1857 |  0.8582 |
| 34 | bbcsp3_negMM_results.mat         | negMM    | 0.2323 | 0.8475 | 0.0049 | 0.0071 | 0.7113 | 0.0043 | 0.0071 |  0.6621 |
| 35 | bbcsp4_negMM_results.mat         | negMM    | 0.1555 | 0.8966 | 0.3918 | 0.3879 | 0.7956 | 0.4301 | 0.3879 |  0.7451 |
| 36 | caltech-20_negMM_results.mat     | negMM    | 0.3015 | 0.7368 | 0.0032 | 0.0067 | 0.5820 | 0.0026 | 0.0067 |  0.3582 |
| 37 | handwritten_negMM_results.mat    | negMM    | 0.1549 | 0.8845 | 0.0059 | 0.0065 | 0.8562 | 0.0055 | 0.0065 |  0.7978 |
| 38 | yahoo_negMM_results.mat          | negMM    | 0.5412 | 0.6551 | 0.0745 | 0.0880 | 0.1889 | 0.1169 | 0.0880 |  0.1361 |

## Summary of the results after the execution of NCF's Stage 2

Setting the path variable:

In [5]:
path = '../stage1_results'

**Build the SRS matrix from a bipartite graph adjacency matrix**

The following step is needed in order to assess the quality of the consensus. Each HDF5 file is opened and the true labels are extracted and stored into a file. This file is then put inside the right folder.

In [18]:
from HDF5Datasets import BBCseg2, BBCseg3, BBCseg4, BBCSPORTSseg2, BBCSPORTSseg3, BBCSPORTSseg4

np.savetxt('../stage1_results/BBCseg2/labels.true', BBCseg2('../data').get_real_labels(), fmt='%d')
np.savetxt('../stage1_results/BBCseg3/labels.true', BBCseg3('../data').get_real_labels(), fmt='%d')
np.savetxt('../stage1_results/BBCseg4/labels.true', BBCseg4('../data').get_real_labels(), fmt='%d')
np.savetxt('../stage1_results/BBCSPORTSseg2/labels.true', BBCSPORTSseg2('../data').get_real_labels(), fmt='%d')
np.savetxt('../stage1_results/BBCSPORTSseg3/labels.true', BBCSPORTSseg3('../data').get_real_labels(), fmt='%d')
np.savetxt('../stage1_results/BBCSPORTSseg4/labels.true', BBCSPORTSseg4('../data').get_real_labels(), fmt='%d')

In [19]:
%%time
embed_and_consensus('BBCseg2', embDim=4, kernelFn='cosine', precomputed_sim=False)

  0%|          | 0/10 [00:00<?, ?it/s]

Opening soft-clustering matrix from ../stage1_results/BBCseg2/simmatrix_DS_Jan072021.205810.out
Storing similarity matrix in sparse format to ../stage1_results/BBCseg2/dotmatrix_DS_Jan072021.205810_csr.npy
../stage1_results/BBCseg2/dotmatrix_DS_Jan072021.205810_csr.npy



 10%|█         | 1/10 [00:00<00:08,  1.12it/s]          

Opening soft-clustering matrix from ../stage1_results/BBCseg2/simmatrix_DS_Jan072021.205815.out
Storing similarity matrix in sparse format to ../stage1_results/BBCseg2/dotmatrix_DS_Jan072021.205815_csr.npy
../stage1_results/BBCseg2/dotmatrix_DS_Jan072021.205815_csr.npy



 20%|██        | 2/10 [00:01<00:07,  1.10it/s]          

...

Opening soft-clustering matrix from ../stage1_results/BBCseg2/simmatrix_DS_Jan072021.205843.out
Storing similarity matrix in sparse format to ../stage1_results/BBCseg2/dotmatrix_DS_Jan072021.205843_csr.npy
../stage1_results/BBCseg2/dotmatrix_DS_Jan072021.205843_csr.npy



 90%|█████████ | 9/10 [00:08<00:00,  1.12it/s]           

Opening soft-clustering matrix from ../stage1_results/BBCseg2/simmatrix_DS_Jan072021.205847.out
Storing similarity matrix in sparse format to ../stage1_results/BBCseg2/dotmatrix_DS_Jan072021.205847_csr.npy
../stage1_results/BBCseg2/dotmatrix_DS_Jan072021.205847_csr.npy



run 9/10:  90%|█████████ | 9/10 [00:00<00:00, 38.98it/s]
                                                        

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.145566 | 0.951789 | 0.148184 | 0.149652 | 0.855236 | 0.146996 | 0.149652 | 0.882567 |
|  1 | 102758375 | 0.145566 | 0.951789 | 0.141015 | 0.140457 | 0.855236 | 0.142005 | 0.140457 | 0.882567 |
|  2 | 102758375 | 0.145566 | 0.951789 | 0.147992 | 0.149901 | 0.855236 | 0.146376 | 0.149901 | 0.882567 |
|  3 | 102758375 | 0.145566 | 0.951789 | 0.269099 | 0.271819 | 0.855236 | 0.266759 | 0.271819 | 0.882567 |
|  4 | 102758375 | 0.145566 | 0.951789 | 0.198403 | 0.200000 | 0.855236 | 0.197110 | 0.200000 | 0.882567 |
|  5 | 102758375 | 0.145566 | 0.951789 | 0.127907 | 0.129026 | 0.855236 | 0.127152 | 0.129026 | 0.882567 |
|  6 | 102758375 | 0.145566 | 0.951789 | 0.144914 | 0.144732 | 0.855236 | 0.145509 | 0.144732 | 0.882567 |
|  7 | 102758375 | 0.145566 | 0.95178

In [20]:
%%time
embed_and_consensus('BBCseg3', embDim=4, kernelFn='cosine', precomputed_sim=False)

  0%|          | 0/10 [00:00<?, ?it/s]

Opening soft-clustering matrix from ../stage1_results/BBCseg3/simmatrix_DS_Jan072021.205927.out
Storing similarity matrix in sparse format to ../stage1_results/BBCseg3/dotmatrix_DS_Jan072021.205927_csr.npy
../stage1_results/BBCseg3/dotmatrix_DS_Jan072021.205927_csr.npy



 10%|█         | 1/10 [00:01<00:10,  1.13s/it]           

Opening soft-clustering matrix from ../stage1_results/BBCseg3/simmatrix_DS_Jan072021.205931.out
Storing similarity matrix in sparse format to ../stage1_results/BBCseg3/dotmatrix_DS_Jan072021.205931_csr.npy
../stage1_results/BBCseg3/dotmatrix_DS_Jan072021.205931_csr.npy



 20%|██        | 2/10 [00:02<00:08,  1.12s/it]           

...

Opening soft-clustering matrix from ../stage1_results/BBCseg3/simmatrix_DS_Jan072021.210002.out
Storing similarity matrix in sparse format to ../stage1_results/BBCseg3/dotmatrix_DS_Jan072021.210002_csr.npy
../stage1_results/BBCseg3/dotmatrix_DS_Jan072021.210002_csr.npy



 90%|█████████ | 9/10 [00:10<00:01,  1.12s/it]           

Opening soft-clustering matrix from ../stage1_results/BBCseg3/simmatrix_DS_Jan072021.210007.out
Storing similarity matrix in sparse format to ../stage1_results/BBCseg3/dotmatrix_DS_Jan072021.210007_csr.npy
../stage1_results/BBCseg3/dotmatrix_DS_Jan072021.210007_csr.npy



run 9/10: 100%|██████████| 10/10 [00:00<00:00, 44.06it/s]
                                                         

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.170893 | 0.944006 | 0.134160 | 0.135568 | 0.828222 | 0.133816 | 0.135568 | 0.863000 |
|  1 | 102758375 | 0.170893 | 0.944006 | 0.166248 | 0.168218 | 0.828222 | 0.165359 | 0.168218 | 0.863000 |
|  2 | 102758375 | 0.170893 | 0.944006 | 0.145749 | 0.146057 | 0.828222 | 0.146526 | 0.146057 | 0.863000 |
|  3 | 102758375 | 0.170893 | 0.944006 | 0.309979 | 0.309700 | 0.828222 | 0.311319 | 0.309700 | 0.863000 |
|  4 | 102758375 | 0.170893 | 0.944006 | 0.166007 | 0.165852 | 0.828222 | 0.167318 | 0.165852 | 0.863000 |
|  5 | 102758375 | 0.170893 | 0.944006 | 0.240162 | 0.243139 | 0.828222 | 0.238333 | 0.243139 | 0.863000 |
|  6 | 102758375 | 0.170893 | 0.944006 | 0.271640 | 0.269953 | 0.828222 | 0.274440 | 0.269953 | 0.863000 |
|  7 | 102758375 | 0.170893 | 0.94400

In [21]:
%%time
embed_and_consensus('BBCseg4', embDim=4, kernelFn='cosine', precomputed_sim=False)

run 4/10:   0%|          | 0/10 [00:00<?, ?it/s]


Opening soft-clustering matrix from ../stage1_results/BBCseg4/simmatrix_DS_Jan072021.210034.out
Storing similarity matrix in sparse format to ../stage1_results/BBCseg4/dotmatrix_DS_Jan072021.210034_csr.npy
../stage1_results/BBCseg4/dotmatrix_DS_Jan072021.210034_csr.npy


run 5/10:   0%|          | 0/10 [00:00<?, ?it/s]


Opening soft-clustering matrix from ../stage1_results/BBCseg4/simmatrix_DS_Jan072021.210038.out
Storing similarity matrix in sparse format to ../stage1_results/BBCseg4/dotmatrix_DS_Jan072021.210038_csr.npy
../stage1_results/BBCseg4/dotmatrix_DS_Jan072021.210038_csr.npy


run 5/10:   0%|          | 0/10 [00:00<?, ?it/s]


...

Opening soft-clustering matrix from ../stage1_results/BBCseg4/simmatrix_DS_Jan072021.210101.out
Storing similarity matrix in sparse format to ../stage1_results/BBCseg4/dotmatrix_DS_Jan072021.210101_csr.npy
../stage1_results/BBCseg4/dotmatrix_DS_Jan072021.210101_csr.npy


run 5/10:   0%|          | 0/10 [00:00<?, ?it/s]

Opening soft-clustering matrix from ../stage1_results/BBCseg4/simmatrix_DS_Jan072021.210104.out
Storing similarity matrix in sparse format to ../stage1_results/BBCseg4/dotmatrix_DS_Jan072021.210104_csr.npy
../stage1_results/BBCseg4/dotmatrix_DS_Jan072021.210104_csr.npy



run 9/10:  60%|██████    | 6/10 [00:00<00:00, 51.72it/s]
                                                        

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.272335 | 0.829635 | 0.270352 | 0.270073 | 0.696347 | 0.300918 | 0.270073 | 0.639560 |
|  1 | 102758375 | 0.272284 | 0.829635 | 0.195772 | 0.184526 | 0.696423 | 0.239145 | 0.184526 | 0.639810 |
|  2 | 102758375 | 0.272001 | 0.829781 | 0.275111 | 0.252409 | 0.696601 | 0.350293 | 0.252409 | 0.639634 |
|  3 | 102758375 | 0.272977 | 0.829489 | 0.216186 | 0.196350 | 0.695459 | 0.280085 | 0.196350 | 0.637694 |
|  4 | 102758375 | 0.271755 | 0.829927 | 0.245179 | 0.225839 | 0.696927 | 0.308811 | 0.225839 | 0.640263 |
|  5 | 102758375 | 0.272523 | 0.829635 | 0.204210 | 0.199854 | 0.696039 | 0.239648 | 0.199854 | 0.638688 |
|  6 | 102758375 | 0.272066 | 0.829781 | 0.185493 | 0.166423 | 0.696498 | 0.241401 | 0.166423 | 0.639137 |
|  7 | 102758375 | 0.270753 | 0.83021

In [22]:
%%time
embed_and_consensus('BBCSPORTSseg2', embDim=4, kernelFn='cosine', precomputed_sim=False)

run 7/10:  60%|██████    | 6/10 [00:00<00:00, 53.31it/s]


Opening soft-clustering matrix from ../stage1_results/BBCSPORTSseg2/simmatrix_DS_Jan072021.205148.out
Storing similarity matrix in sparse format to ../stage1_results/BBCSPORTSseg2/dotmatrix_DS_Jan072021.205148_csr.npy
../stage1_results/BBCSPORTSseg2/dotmatrix_DS_Jan072021.205148_csr.npy


run 6/10:  50%|█████     | 5/10 [00:00<00:00, 48.07it/s]


Opening soft-clustering matrix from ../stage1_results/BBCSPORTSseg2/simmatrix_DS_Jan072021.205149.out
Storing similarity matrix in sparse format to ../stage1_results/BBCSPORTSseg2/dotmatrix_DS_Jan072021.205149_csr.npy
../stage1_results/BBCSPORTSseg2/dotmatrix_DS_Jan072021.205149_csr.npy


run 7/10:  60%|██████    | 6/10 [00:00<00:00, 55.61it/s]


...

Opening soft-clustering matrix from ../stage1_results/BBCSPORTSseg2/simmatrix_DS_Jan072021.205157.out
Storing similarity matrix in sparse format to ../stage1_results/BBCSPORTSseg2/dotmatrix_DS_Jan072021.205157_csr.npy
../stage1_results/BBCSPORTSseg2/dotmatrix_DS_Jan072021.205157_csr.npy


run 7/10:  60%|██████    | 6/10 [00:00<00:00, 58.10it/s]


Opening soft-clustering matrix from ../stage1_results/BBCSPORTSseg2/simmatrix_DS_Jan072021.205158.out
Storing similarity matrix in sparse format to ../stage1_results/BBCSPORTSseg2/dotmatrix_DS_Jan072021.205158_csr.npy
../stage1_results/BBCSPORTSseg2/dotmatrix_DS_Jan072021.205158_csr.npy


run 9/10:  60%|██████    | 6/10 [00:00<00:00, 58.10it/s]
                                                        

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.129860 | 0.954044 | 0.223724 | 0.226838 | 0.866411 | 0.221836 | 0.226838 | 0.880003 |
|  1 | 102758375 | 0.129860 | 0.954044 | 0.081359 | 0.084191 | 0.866411 | 0.079565 | 0.084191 | 0.880003 |
|  2 | 102758375 | 0.129860 | 0.954044 | 0.080564 | 0.082353 | 0.866411 | 0.080176 | 0.082353 | 0.880003 |
|  3 | 102758375 | 0.129860 | 0.954044 | 0.188464 | 0.191176 | 0.866411 | 0.187058 | 0.191176 | 0.880003 |
|  4 | 102758375 | 0.129860 | 0.954044 | 0.014545 | 0.015625 | 0.866411 | 0.014482 | 0.015625 | 0.880003 |
|  5 | 102758375 | 0.129860 | 0.954044 | 0.176031 | 0.179412 | 0.866411 | 0.173883 | 0.179412 | 0.880003 |
|  6 | 102758375 | 0.129860 | 0.954044 | 0.167267 | 0.169853 | 0.866411 | 0.166049 | 0.169853 | 0.880003 |
|  7 | 102758375 | 0.129860 | 0.95404

In [23]:
%%time
embed_and_consensus('BBCSPORTSseg3', embDim=4, kernelFn='cosine', precomputed_sim=False)

 10%|█         | 1/10 [00:00<00:01,  5.41it/s]          

Opening soft-clustering matrix from ../stage1_results/BBCSPORTSseg3/simmatrix_DS_Jan072021.205310.out
Storing similarity matrix in sparse format to ../stage1_results/BBCSPORTSseg3/dotmatrix_DS_Jan072021.205310_csr.npy
../stage1_results/BBCSPORTSseg3/dotmatrix_DS_Jan072021.205310_csr.npy
Opening soft-clustering matrix from ../stage1_results/BBCSPORTSseg3/simmatrix_DS_Jan072021.205311.out
Storing similarity matrix in sparse format to ../stage1_results/BBCSPORTSseg3/dotmatrix_DS_Jan072021.205311_csr.npy


run 9/10:  60%|██████    | 6/10 [00:00<00:00, 51.71it/s]



../stage1_results/BBCSPORTSseg3/dotmatrix_DS_Jan072021.205311_csr.npy


run 9/10:  60%|██████    | 6/10 [00:00<00:00, 53.57it/s]

...

run 8/10:  60%|██████    | 6/10 [00:00<00:00, 58.25it/s]


Opening soft-clustering matrix from ../stage1_results/BBCSPORTSseg3/simmatrix_DS_Jan072021.205318.out
Storing similarity matrix in sparse format to ../stage1_results/BBCSPORTSseg3/dotmatrix_DS_Jan072021.205318_csr.npy
../stage1_results/BBCSPORTSseg3/dotmatrix_DS_Jan072021.205318_csr.npy


run 9/10:  60%|██████    | 6/10 [00:00<00:00, 55.56it/s]

Opening soft-clustering matrix from ../stage1_results/BBCSPORTSseg3/simmatrix_DS_Jan072021.205320.out
Storing similarity matrix in sparse format to ../stage1_results/BBCSPORTSseg3/dotmatrix_DS_Jan072021.205320_csr.npy
../stage1_results/BBCSPORTSseg3/dotmatrix_DS_Jan072021.205320_csr.npy


|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.299861 | 0.822695 | 0.289235 | 0.282270 | 0.643904 | 0.321957 | 0.282270 | 0.587411 |
|  1 | 102758375 | 0.299861 | 0.822695 | 0.174464 | 0.169149 | 0.643904 | 0.216177 | 0.169149 | 0.587411 |
|  2 | 102758375 | 0.299861 | 0.822695 | 0.289998 | 0.271986 | 0.643904 | 0.353610 | 0.271986 | 0.587411 |
|  3 | 102758375 | 0.299861 | 0.822695 | 0.208952 | 0.198227 | 0.643904 | 0.260784 | 0.198227 | 0.587411 |
|  4 | 102758375 | 0.299861 | 0.822695 | 0.217026 | 0.208865 | 0.643904 | 0.276094 | 0.208865 | 0.587411 |
|  5 | 102758375 | 0.299861 | 0.822695 | 0.163258 | 0.165957 | 0.643904 | 0.182054 | 0.165957 | 0.587411 |
|  6 | 102758375 | 0.299861 | 0.822695 | 0.170438 | 0.168440 | 0.643904 | 0.191221 | 0.168440 | 0.587411 |
|  7 | 102758375 | 0.299861 | 0.82269

In [24]:
%%time
embed_and_consensus('BBCSPORTSseg4', embDim=4, kernelFn='cosine', precomputed_sim=False)

run 9/10:  60%|██████    | 6/10 [00:00<00:00, 57.69it/s]

Opening soft-clustering matrix from ../stage1_results/BBCSPORTSseg4/simmatrix_DS_Jan072021.205343.out
Storing similarity matrix in sparse format to ../stage1_results/BBCSPORTSseg4/dotmatrix_DS_Jan072021.205343_csr.npy
../stage1_results/BBCSPORTSseg4/dotmatrix_DS_Jan072021.205343_csr.npy



run 9/10:  60%|██████    | 6/10 [00:00<00:00, 52.62it/s]

Opening soft-clustering matrix from ../stage1_results/BBCSPORTSseg4/simmatrix_DS_Jan072021.205344.out
Storing similarity matrix in sparse format to ../stage1_results/BBCSPORTSseg4/dotmatrix_DS_Jan072021.205344_csr.npy
../stage1_results/BBCSPORTSseg4/dotmatrix_DS_Jan072021.205344_csr.npy



 43%|████▎     | 3/7 [00:00<00:00,  5.42it/s]           

...

Opening soft-clustering matrix from ../stage1_results/BBCSPORTSseg4/simmatrix_DS_Jan072021.205346.out
Storing similarity matrix in sparse format to ../stage1_results/BBCSPORTSseg4/dotmatrix_DS_Jan072021.205346_csr.npy
../stage1_results/BBCSPORTSseg4/dotmatrix_DS_Jan072021.205346_csr.npy
Opening soft-clustering matrix from ../stage1_results/BBCSPORTSseg4/simmatrix_DS_Jan072021.205347.out
Storing similarity matrix in sparse format to ../stage1_results/BBCSPORTSseg4/dotmatrix_DS_Jan072021.205347_csr.npy
../stage1_results/BBCSPORTSseg4/dotmatrix_DS_Jan072021.205347_csr.npy


run 9/10:  70%|███████   | 7/10 [00:00<00:00, 61.95it/s]


Opening soft-clustering matrix from ../stage1_results/BBCSPORTSseg4/simmatrix_DS_Jan072021.205348.out
Storing similarity matrix in sparse format to ../stage1_results/BBCSPORTSseg4/dotmatrix_DS_Jan072021.205348_csr.npy
../stage1_results/BBCSPORTSseg4/dotmatrix_DS_Jan072021.205348_csr.npy


run 9/10:  60%|██████    | 6/10 [00:00<00:00, 59.98it/s]
                                                        

Opening soft-clustering matrix from ../stage1_results/BBCSPORTSseg4/simmatrix_DS_Jan072021.205349.out
Storing similarity matrix in sparse format to ../stage1_results/BBCSPORTSseg4/dotmatrix_DS_Jan072021.205349_csr.npy
../stage1_results/BBCSPORTSseg4/dotmatrix_DS_Jan072021.205349_csr.npy
|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.343413 | 0.758621 | 0.240015 | 0.222414 | 0.613757 | 0.334101 | 0.222414 | 0.451915 |
|  1 | 102758375 | 0.323478 | 0.782759 | 0.316779 | 0.306034 | 0.619889 | 0.363055 | 0.306034 | 0.517628 |
|  2 | 102758375 | 0.360693 | 0.758621 | 0.107754 | 0.114655 | 0.588527 | 0.128216 | 0.114655 | 0.444115 |
|  3 | 102758375 | 0.353289 | 0.753448 | 0.182676 | 0.166379 | 0.602460 | 0.273635 | 0.166379 | 0.441108 |
|  4 | 102758375 | 0.360693 | 0.758621 | 0.183596 | 0.179310 | 0.58852

**Check the first ones and then delete the old bbc datasets**

In [9]:
%%time
embed_and_consensus('BBC-seg2', embDim=4, kernelFn='cosine', precomputed_sim=False)

  0%|          | 0/10 [00:00<?, ?it/s]

Opening soft-clustering matrix from ../stage1_results/BBC-seg2/simmatrix_DS_Jan022021.170037.out
Storing similarity matrix in sparse format to ../stage1_results/BBC-seg2/dotmatrix_DS_Jan022021.170037_csr.npy
../stage1_results/BBC-seg2/dotmatrix_DS_Jan022021.170037_csr.npy



 10%|█         | 1/10 [00:04<00:37,  4.16s/it]          

Opening soft-clustering matrix from ../stage1_results/BBC-seg2/simmatrix_DS_Jan022021.170041.out
Storing similarity matrix in sparse format to ../stage1_results/BBC-seg2/dotmatrix_DS_Jan022021.170041_csr.npy
../stage1_results/BBC-seg2/dotmatrix_DS_Jan022021.170041_csr.npy



 20%|██        | 2/10 [00:08<00:33,  4.19s/it]          

...

Opening soft-clustering matrix from ../stage1_results/BBC-seg2/simmatrix_DS_Jan022021.170110.out
Storing similarity matrix in sparse format to ../stage1_results/BBC-seg2/dotmatrix_DS_Jan022021.170110_csr.npy
../stage1_results/BBC-seg2/dotmatrix_DS_Jan022021.170110_csr.npy



 90%|█████████ | 9/10 [00:37<00:04,  4.12s/it]          

Opening soft-clustering matrix from ../stage1_results/BBC-seg2/simmatrix_DS_Jan022021.170115.out
Storing similarity matrix in sparse format to ../stage1_results/BBC-seg2/dotmatrix_DS_Jan022021.170115_csr.npy
../stage1_results/BBC-seg2/dotmatrix_DS_Jan022021.170115_csr.npy



run 9/10:  80%|████████  | 8/10 [00:00<00:00, 36.23it/s]


|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.149259 | 0.951044 | 0.247001 | 0.246521 | 0.851238 | 0.248031 | 0.246521 | 0.881266 |
|  1 | 102758375 | 0.150560 | 0.950497 | 0.209846 | 0.212127 | 0.849929 | 0.207888 | 0.212127 | 0.880068 |
|  2 | 102758375 | 0.152221 | 0.949801 | 0.128924 | 0.129622 | 0.848261 | 0.128573 | 0.129622 | 0.878522 |
|  3 | 102758375 | 0.149619 | 0.950895 | 0.168596 | 0.170278 | 0.850880 | 0.167175 | 0.170278 | 0.880918 |
|  4 | 102758375 | 0.149259 | 0.951044 | 0.196355 | 0.196769 | 0.851238 | 0.196322 | 0.196769 | 0.881266 |
|  5 | 102758375 | 0.152221 | 0.949801 | 0.239271 | 0.240606 | 0.848261 | 0.238347 | 0.240606 | 0.878522 |
|  6 | 102758375 | 0.150920 | 0.950348 | 0.262248 | 0.262425 | 0.849570 | 0.262516 | 0.262425 | 0.879720 |
|  7 | 102758375 | 0.152221 | 0.94980

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.149259 | 0.951044 | 0.247001 | 0.246521 | 0.851238 | 0.248031 | 0.246521 | 0.881266 |
|  1 | 102758375 | 0.150560 | 0.950497 | 0.209846 | 0.212127 | 0.849929 | 0.207888 | 0.212127 | 0.880068 |
|  2 | 102758375 | 0.152221 | 0.949801 | 0.128924 | 0.129622 | 0.848261 | 0.128573 | 0.129622 | 0.878522 |
|  3 | 102758375 | 0.149619 | 0.950895 | 0.168596 | 0.170278 | 0.850880 | 0.167175 | 0.170278 | 0.880918 |
|  4 | 102758375 | 0.149259 | 0.951044 | 0.196355 | 0.196769 | 0.851238 | 0.196322 | 0.196769 | 0.881266 |
|  5 | 102758375 | 0.152221 | 0.949801 | 0.239271 | 0.240606 | 0.848261 | 0.238347 | 0.240606 | 0.878522 |
|  6 | 102758375 | 0.150920 | 0.950348 | 0.262248 | 0.262425 | 0.849570 | 0.262516 | 0.262425 | 0.879720 |
|  7 | 102758375 | 0.152221 | 0.949801 | 0.265026 | 0.266849 | 0.848261 | 0.263596 | 0.266849 | 0.878522 |
|  8 | 102758375 | 0.152221 | 0.949801 | 0.161928 | 0.162376 | 0.848261 | 0.161830 | 0.162376 | 0.878522 |
|  9 | 102758375 | 0.149259 | 0.951044 | 0.101166 | 0.101143 | 0.851238 | 0.101497 | 0.101143 | 0.881266 |
| 10 | Average   | 0.150776 | 0.950408 | 0.198036 | 0.198872 | 0.849714 | 0.197578 | 0.198872 | 0.879860 |

In [10]:
%%time
embed_and_consensus('BBC-seg3', embDim=4, kernelFn='cosine', precomputed_sim=False)

  0%|          | 0/10 [00:00<?, ?it/s]

Opening soft-clustering matrix from ../stage1_results/BBC-seg3/simmatrix_DS_Jan022021.171003.out
Storing similarity matrix in sparse format to ../stage1_results/BBC-seg3/dotmatrix_DS_Jan022021.171003_csr.npy
../stage1_results/BBC-seg3/dotmatrix_DS_Jan022021.171003_csr.npy



 10%|█         | 1/10 [00:00<00:04,  1.98it/s]           

Opening soft-clustering matrix from ../stage1_results/BBC-seg3/simmatrix_DS_Jan022021.171008.out
Storing similarity matrix in sparse format to ../stage1_results/BBC-seg3/dotmatrix_DS_Jan022021.171008_csr.npy
../stage1_results/BBC-seg3/dotmatrix_DS_Jan022021.171008_csr.npy



 20%|██        | 2/10 [00:01<00:03,  2.00it/s]           

...

Opening soft-clustering matrix from ../stage1_results/BBC-seg3/simmatrix_DS_Jan022021.171037.out
Storing similarity matrix in sparse format to ../stage1_results/BBC-seg3/dotmatrix_DS_Jan022021.171037_csr.npy
../stage1_results/BBC-seg3/dotmatrix_DS_Jan022021.171037_csr.npy



 90%|█████████ | 9/10 [00:04<00:00,  2.20it/s]           

Opening soft-clustering matrix from ../stage1_results/BBC-seg3/simmatrix_DS_Jan022021.171041.out
Storing similarity matrix in sparse format to ../stage1_results/BBC-seg3/dotmatrix_DS_Jan022021.171041_csr.npy
../stage1_results/BBC-seg3/dotmatrix_DS_Jan022021.171041_csr.npy



run 9/10: 100%|██████████| 10/10 [00:00<00:00, 44.83it/s]
                                                         

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.147757 | 0.951893 | 0.420682 | 0.420110 | 0.850715 | 0.422040 | 0.420110 | 0.881513 |
|  1 | 102758375 | 0.147565 | 0.951972 | 0.314141 | 0.314511 | 0.850922 | 0.314697 | 0.314511 | 0.881687 |
|  2 | 102758375 | 0.147757 | 0.951893 | 0.152430 | 0.155205 | 0.850715 | 0.150422 | 0.155205 | 0.881513 |
|  3 | 102758375 | 0.147757 | 0.951893 | 0.237988 | 0.236041 | 0.850715 | 0.240757 | 0.236041 | 0.881513 |
|  4 | 102758375 | 0.147757 | 0.951893 | 0.179359 | 0.179259 | 0.850715 | 0.180248 | 0.179259 | 0.881513 |
|  5 | 102758375 | 0.147757 | 0.951893 | 0.244366 | 0.243375 | 0.850715 | 0.246283 | 0.243375 | 0.881513 |
|  6 | 102758375 | 0.147757 | 0.951893 | 0.190005 | 0.192823 | 0.850715 | 0.187894 | 0.192823 | 0.881513 |
|  7 | 102758375 | 0.147757 | 0.95189

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.147757 | 0.951893 | 0.420682 | 0.420110 | 0.850715 | 0.422040 | 0.420110 | 0.881513 |
|  1 | 102758375 | 0.147565 | 0.951972 | 0.314141 | 0.314511 | 0.850922 | 0.314697 | 0.314511 | 0.881687 |
|  2 | 102758375 | 0.147757 | 0.951893 | 0.152430 | 0.155205 | 0.850715 | 0.150422 | 0.155205 | 0.881513 |
|  3 | 102758375 | 0.147757 | 0.951893 | 0.237988 | 0.236041 | 0.850715 | 0.240757 | 0.236041 | 0.881513 |
|  4 | 102758375 | 0.147757 | 0.951893 | 0.179359 | 0.179259 | 0.850715 | 0.180248 | 0.179259 | 0.881513 |
|  5 | 102758375 | 0.147757 | 0.951893 | 0.244366 | 0.243375 | 0.850715 | 0.246283 | 0.243375 | 0.881513 |
|  6 | 102758375 | 0.147757 | 0.951893 | 0.190005 | 0.192823 | 0.850715 | 0.187894 | 0.192823 | 0.881513 |
|  7 | 102758375 | 0.147757 | 0.951893 | 0.195349 | 0.197161 | 0.850715 | 0.194249 | 0.197161 | 0.881513 |
|  8 | 102758375 | 0.147757 | 0.951893 | 0.200190 | 0.199685 | 0.850715 | 0.201537 | 0.199685 | 0.881513 |
|  9 | 102758375 | 0.147757 | 0.951893 | 0.235219 | 0.235095 | 0.850715 | 0.236249 | 0.235095 | 0.881513 |
| 10 | Average   | 0.147738 | 0.951901 | 0.236973 | 0.237326 | 0.850735 | 0.237437 | 0.237326 | 0.881530 |

In [6]:
%%time
embed_and_consensus('Reuters', embDim=4, kernelFn='cosine', precomputed_sim=False)

  0%|          | 0/10 [00:00<?, ?it/s]

Opening soft-clustering matrix from ../stage1_results/Reuters/simmatrix_DS_Jan022021.224759.out
Storing similarity matrix in sparse format to ../stage1_results/Reuters/dotmatrix_DS_Jan022021.224759_csr.npy
../stage1_results/Reuters/dotmatrix_DS_Jan022021.224759_csr.npy


MemoryError: 

In [8]:
%%time
perform_evaluate_runs('BBC-seg4', embDim=4, kernelFn='cosine')

100%|██████████| 10/10 [00:03<00:00,  2.91it/s]

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.183388 | 0.932847 | 0.305188 | 0.304526 | 0.807615 | 0.309141 | 0.304526 | 0.835237 |
|  1 | 102758375 | 0.181714 | 0.933285 | 0.098665 | 0.100000 | 0.809284 | 0.101374 | 0.100000 | 0.836075 |
|  2 | 102758375 | 0.183388 | 0.932847 | 0.120351 | 0.122336 | 0.807615 | 0.120372 | 0.122336 | 0.835237 |
|  3 | 102758375 | 0.184038 | 0.932555 | 0.139719 | 0.142774 | 0.806961 | 0.141232 | 0.142774 | 0.834496 |
|  4 | 102758375 | 0.182830 | 0.932993 | 0.116874 | 0.117664 | 0.808172 | 0.120214 | 0.117664 | 0.835517 |
|  5 | 102758375 | 0.183388 | 0.932847 | 0.161820 | 0.164088 | 0.807615 | 0.162908 | 0.164088 | 0.835237 |
|  6 | 102758375 | 0.182830 | 0.932993 | 0.090352 | 0.090219 | 0.808172 | 0.094477 | 0.090219 | 0.835517 |
|  7 | 102758375 | 0.182830 | 0.93299

dim:4, cosine

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.183388 | 0.932847 | 0.305188 | 0.304526 | 0.807615 | 0.309141 | 0.304526 | 0.835237 |
|  1 | 102758375 | 0.181714 | 0.933285 | 0.098665 | 0.100000 | 0.809284 | 0.101374 | 0.100000 | 0.836075 |
|  2 | 102758375 | 0.183388 | 0.932847 | 0.120351 | 0.122336 | 0.807615 | 0.120372 | 0.122336 | 0.835237 |
|  3 | 102758375 | 0.184038 | 0.932555 | 0.139719 | 0.142774 | 0.806961 | 0.141232 | 0.142774 | 0.834496 |
|  4 | 102758375 | 0.182830 | 0.932993 | 0.116874 | 0.117664 | 0.808172 | 0.120214 | 0.117664 | 0.835517 |
|  5 | 102758375 | 0.183388 | 0.932847 | 0.161820 | 0.164088 | 0.807615 | 0.162908 | 0.164088 | 0.835237 |
|  6 | 102758375 | 0.182830 | 0.932993 | 0.090352 | 0.090219 | 0.808172 | 0.094477 | 0.090219 | 0.835517 |
|  7 | 102758375 | 0.182830 | 0.932993 | 0.138518 | 0.140146 | 0.808172 | 0.140495 | 0.140146 | 0.835517 |
|  8 | 102758375 | 0.181985 | 0.933139 | 0.171865 | 0.174891 | 0.808953 | 0.173101 | 0.174891 | 0.835747 |
|  9 | 102758375 | 0.184038 | 0.932555 | 0.208822 | 0.211095 | 0.806961 | 0.210462 | 0.211095 | 0.834496 |
| 10 | Average   | 0.183043 | 0.932905 | 0.155217 | 0.156774 | 0.807952 | 0.157378 | 0.156774 | 0.835308 |
CPU times: user 8.88 s, sys: 4.01 s, total: 12.9 s
Wall time: 2.95 s



|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.268472 | 0.894161 | 0.149339 | 0.155328 | 0.721357 | 0.148146 | 0.155328 | 0.755605 |
|  1 | 102758375 | 0.268042 | 0.894307 | 0.094929 | 0.097664 | 0.721754 | 0.098474 | 0.097664 | 0.755752 |
|  2 | 102758375 | 0.269831 | 0.893431 | 0.118097 | 0.124380 | 0.720049 | 0.117356 | 0.124380 | 0.754138 |
|  3 | 102758375 | 0.269305 | 0.893723 | 0.210728 | 0.215182 | 0.720558 | 0.211137 | 0.215182 | 0.754575 |
|  4 | 102758375 | 0.266446 | 0.895182 | 0.153666 | 0.154599 | 0.723294 | 0.158414 | 0.154599 | 0.757712 |
|  5 | 102758375 | 0.267779 | 0.894453 | 0.135532 | 0.139416 | 0.722008 | 0.136482 | 0.139416 | 0.755971 |
|  6 | 102758375 | 0.268304 | 0.894307 | 0.133509 | 0.138832 | 0.721531 | 0.132912 | 0.138832 | 0.755996 |
|  7 | 102758375 | 0.267542 | 0.894599 | 0.146864 | 0.150949 | 0.722240 | 0.148295 | 0.150949 | 0.756463 |
|  8 | 102758375 | 0.272487 | 0.890219 | 0.300385 | 0.306861 | 0.716485 | 0.301102 | 0.306861 | 0.749963 |
|  9 | 102758375 | 0.268972 | 0.893869 | 0.039151 | 0.043942 | 0.720872 | 0.040679 | 0.043942 | 0.754894 |
| 10 | Average   | 0.268718 | 0.893825 | 0.148220 | 0.152715 | 0.721015 | 0.149300 | 0.152715 | 0.755107 |

CPU times: user 4min 56s, sys: 2.46 s, total: 4min 58s
Wall time: 4min 46s

In [13]:
%%time
perform_evaluate_runs('Caltech', embDim=8, kernelFn='cosine')

100%|██████████| 10/10 [00:11<00:00,  1.10s/it]

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.428383 | 0.659640 | 0.043620 | 0.044007 | 0.436066 | 0.073938 | 0.044007 | 0.303957 |
|  1 | 102758375 | 0.427989 | 0.662825 | 0.065009 | 0.058256 | 0.436709 | 0.097686 | 0.058256 | 0.318946 |
|  2 | 102758375 | 0.418265 | 0.652850 | 0.113960 | 0.106873 | 0.450076 | 0.199417 | 0.106873 | 0.346506 |
|  3 | 102758375 | 0.419574 | 0.652096 | 0.068019 | 0.066345 | 0.444782 | 0.104811 | 0.066345 | 0.336888 |
|  4 | 102758375 | 0.429605 | 0.658466 | 0.051974 | 0.055532 | 0.436864 | 0.076387 | 0.055532 | 0.318954 |
|  5 | 102758375 | 0.428818 | 0.664878 | 0.062528 | 0.059137 | 0.439885 | 0.154630 | 0.059137 | 0.314035 |
|  6 | 102758375 | 0.425140 | 0.645013 | 0.048215 | 0.044593 | 0.439948 | 0.109057 | 0.044593 | 0.335990 |
|  7 | 102758375 | 0.428611 | 0.66311

dim:8, cosine

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.428383 | 0.659640 | 0.043620 | 0.044007 | 0.436066 | 0.073938 | 0.044007 | 0.303957 |
|  1 | 102758375 | 0.427989 | 0.662825 | 0.065009 | 0.058256 | 0.436709 | 0.097686 | 0.058256 | 0.318946 |
|  2 | 102758375 | 0.418265 | 0.652850 | 0.113960 | 0.106873 | 0.450076 | 0.199417 | 0.106873 | 0.346506 |
|  3 | 102758375 | 0.419574 | 0.652096 | 0.068019 | 0.066345 | 0.444782 | 0.104811 | 0.066345 | 0.336888 |
|  4 | 102758375 | 0.429605 | 0.658466 | 0.051974 | 0.055532 | 0.436864 | 0.076387 | 0.055532 | 0.318954 |
|  5 | 102758375 | 0.428818 | 0.664878 | 0.062528 | 0.059137 | 0.439885 | 0.154630 | 0.059137 | 0.314035 |
|  6 | 102758375 | 0.425140 | 0.645013 | 0.048215 | 0.044593 | 0.439948 | 0.109057 | 0.044593 | 0.335990 |
|  7 | 102758375 | 0.428611 | 0.663118 | 0.057425 | 0.056370 | 0.439868 | 0.097537 | 0.056370 | 0.314683 |
|  8 | 102758375 | 0.437024 | 0.656035 | 0.049690 | 0.044677 | 0.430078 | 0.095058 | 0.044677 | 0.333822 |
|  9 | 102758375 | 0.415802 | 0.653353 | 0.033990 | 0.034996 | 0.449886 | 0.052390 | 0.034996 | 0.351205 |
| 10 | Average   | 0.425921 | 0.656827 | 0.059443 | 0.057079 | 0.440416 | 0.106091 | 0.057079 | 0.327498 |
CPU times: user 31.9 s, sys: 9.97 s, total: 41.9 s
Wall time: 11 s


dim:5 , cosine

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.429888 | 0.648910 | 0.052559 | 0.052096 | 0.425421 | 0.078130 | 0.052096 | 0.322740 |
|  1 | 102758375 | 0.430596 | 0.649581 | 0.050770 | 0.046773 | 0.423121 | 0.094186 | 0.046773 | 0.316401 |
|  2 | 102758375 | 0.405382 | 0.659514 | 0.094435 | 0.085247 | 0.450282 | 0.159397 | 0.085247 | 0.354355 |
|  3 | 102758375 | 0.407962 | 0.655448 | 0.073609 | 0.064920 | 0.448392 | 0.125081 | 0.064920 | 0.360443 |
|  4 | 102758375 | 0.430999 | 0.648701 | 0.023385 | 0.027494 | 0.420953 | 0.038343 | 0.027494 | 0.309670 |
|  5 | 102758375 | 0.431033 | 0.650838 | 0.057918 | 0.048868 | 0.422462 | 0.109296 | 0.048868 | 0.314567 |
|  6 | 102758375 | 0.409791 | 0.655029 | 0.064198 | 0.054862 | 0.445235 | 0.138087 | 0.054862 | 0.348076 |
|  7 | 102758375 | 0.429922 | 0.648868 | 0.058080 | 0.049162 | 0.425611 | 0.120052 | 0.049162 | 0.328405 |
|  8 | 102758375 | 0.422801 | 0.661065 | 0.071160 | 0.065423 | 0.433804 | 0.118065 | 0.065423 | 0.350798 |
|  9 | 102758375 | 0.407643 | 0.656329 | 0.065358 | 0.062531 | 0.448731 | 0.142431 | 0.062531 | 0.349878 |
| 10 | Average   | 0.420602 | 0.653428 | 0.061147 | 0.055738 | 0.434401 | 0.112307 | 0.055738 | 0.335533 |
CPU times: user 31.4 s, sys: 9.99 s, total: 41.4 s
Wall time: 10.9 s

dim:3, cosine

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.429080 | 0.649246 | 0.030382 | 0.031140 | 0.425903 | 0.045765 | 0.031140 | 0.350819 |
|  1 | 102758375 | 0.430828 | 0.649329 | 0.034217 | 0.034367 | 0.423641 | 0.051573 | 0.034367 | 0.345043 |
|  2 | 102758375 | 0.414870 | 0.643713 | 0.093608 | 0.080134 | 0.433554 | 0.189920 | 0.080134 | 0.312642 |
|  3 | 102758375 | 0.415314 | 0.643588 | 0.055367 | 0.047946 | 0.432987 | 0.114854 | 0.047946 | 0.312657 |
|  4 | 102758375 | 0.427223 | 0.650168 | 0.074751 | 0.065130 | 0.427829 | 0.117880 | 0.065130 | 0.352501 |
|  5 | 102758375 | 0.429397 | 0.648156 | 0.040202 | 0.037427 | 0.422792 | 0.073292 | 0.037427 | 0.339596 |
|  6 | 102758375 | 0.417945 | 0.643588 | 0.058441 | 0.053646 | 0.428766 | 0.116574 | 0.053646 | 0.299925 |
|  7 | 102758375 | 0.429430 | 0.650754 | 0.049732 | 0.045851 | 0.427227 | 0.093415 | 0.045851 | 0.357674 |
|  8 | 102758375 | 0.428885 | 0.648659 | 0.057716 | 0.059262 | 0.420952 | 0.088058 | 0.059262 | 0.325333 |
|  9 | 102758375 | 0.414440 | 0.643965 | 0.068801 | 0.054694 | 0.432413 | 0.199044 | 0.054694 | 0.310104 |
| 10 | Average   | 0.423741 | 0.647117 | 0.056322 | 0.050960 | 0.427606 | 0.109038 | 0.050960 | 0.330629 |
CPU times: user 31.9 s, sys: 9.51 s, total: 41.4 s
Wall time: 11.2 s


dim : 3

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.445440 | 0.643336 | 0.026973 | 0.027033 | 0.414287 | 0.055858 | 0.027033 | 0.342926 |
|  1 | 102758375 | 0.444238 | 0.643336 | 0.045111 | 0.041408 | 0.416247 | 0.070447 | 0.041408 | 0.348293 |
|  2 | 102758375 | 0.424884 | 0.648198 | 0.077482 | 0.071961 | 0.426085 | 0.128635 | 0.071961 | 0.304730 |
|  3 | 102758375 | 0.423435 | 0.648198 | 0.077158 | 0.065339 | 0.426261 | 0.186978 | 0.065339 | 0.301516 |
|  4 | 102758375 | 0.443114 | 0.643168 | 0.046479 | 0.041869 | 0.415925 | 0.073534 | 0.041869 | 0.346467 |
|  5 | 102758375 | 0.441940 | 0.644216 | 0.027960 | 0.033445 | 0.415597 | 0.042074 | 0.033445 | 0.339148 |
|  6 | 102758375 | 0.422699 | 0.645306 | 0.045728 | 0.044216 | 0.426639 | 0.077841 | 0.044216 | 0.304276 |
|  7 | 102758375 | 0.443025 | 0.644258 | 0.057770 | 0.052054 | 0.419893 | 0.104900 | 0.052054 | 0.376782 |
|  8 | 102758375 | 0.444823 | 0.644593 | 0.035458 | 0.036421 | 0.411878 | 0.068847 | 0.036421 | 0.349812 |
|  9 | 102758375 | 0.422160 | 0.648617 | 0.055328 | 0.041953 | 0.428452 | 0.199043 | 0.041953 | 0.308407 |
| 10 | Average   | 0.435576 | 0.645323 | 0.049545 | 0.045570 | 0.420126 | 0.100816 | 0.045570 | 0.332236 |

CPU times: user 1min 46s, sys: 57.2 s, total: 2min 44s
Wall time: 28.2 s

In [38]:
%%time
perform_evaluate_runs('handwritten', embDim=6, kernelFn='cosine') # no important difference with the default thresold

100%|██████████| 10/10 [00:07<00:00,  1.40it/s]

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.281818 | 0.761050 | 0.095272 | 0.099400 | 0.730629 | 0.101884 | 0.099400 | 0.627191 |
|  1 | 102758375 | 0.276767 | 0.770700 | 0.069029 | 0.077700 | 0.734134 | 0.066716 | 0.077700 | 0.637387 |
|  2 | 102758375 | 0.277524 | 0.764200 | 0.138228 | 0.145250 | 0.733049 | 0.142899 | 0.145250 | 0.636140 |
|  3 | 102758375 | 0.278512 | 0.770100 | 0.119809 | 0.123950 | 0.732739 | 0.125409 | 0.123950 | 0.632471 |
|  4 | 102758375 | 0.278240 | 0.764150 | 0.095165 | 0.102700 | 0.732373 | 0.098505 | 0.102700 | 0.630341 |
|  5 | 102758375 | 0.284441 | 0.763200 | 0.074636 | 0.077600 | 0.729158 | 0.077518 | 0.077600 | 0.619431 |
|  6 | 102758375 | 0.275399 | 0.774150 | 0.118619 | 0.124000 | 0.734071 | 0.118853 | 0.124000 | 0.640756 |
|  7 | 102758375 | 0.302031 | 0.74215

dim:6, cosine 

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.281818 | 0.761050 | 0.095272 | 0.099400 | 0.730629 | 0.101884 | 0.099400 | 0.627191 |
|  1 | 102758375 | 0.276767 | 0.770700 | 0.069029 | 0.077700 | 0.734134 | 0.066716 | 0.077700 | 0.637387 |
|  2 | 102758375 | 0.277524 | 0.764200 | 0.138228 | 0.145250 | 0.733049 | 0.142899 | 0.145250 | 0.636140 |
|  3 | 102758375 | 0.278512 | 0.770100 | 0.119809 | 0.123950 | 0.732739 | 0.125409 | 0.123950 | 0.632471 |
|  4 | 102758375 | 0.278240 | 0.764150 | 0.095165 | 0.102700 | 0.732373 | 0.098505 | 0.102700 | 0.630341 |
|  5 | 102758375 | 0.284441 | 0.763200 | 0.074636 | 0.077600 | 0.729158 | 0.077518 | 0.077600 | 0.619431 |
|  6 | 102758375 | 0.275399 | 0.774150 | 0.118619 | 0.124000 | 0.734071 | 0.118853 | 0.124000 | 0.640756 |
|  7 | 102758375 | 0.302031 | 0.742150 | 0.116881 | 0.123550 | 0.713664 | 0.121963 | 0.123550 | 0.605184 |
|  8 | 102758375 | 0.277821 | 0.765100 | 0.100481 | 0.112100 | 0.732227 | 0.098190 | 0.112100 | 0.634336 |
|  9 | 102758375 | 0.279416 | 0.768800 | 0.075536 | 0.075800 | 0.732158 | 0.077696 | 0.075800 | 0.627847 |
| 10 | Average   | 0.281197 | 0.764360 | 0.100366 | 0.106205 | 0.730420 | 0.102963 | 0.106205 | 0.629108 |
CPU times: user 21.4 s, sys: 7.06 s, total: 28.5 s
Wall time: 7.18 s

dim : 6

|    | seed      |        E |        P |       F1 |      ACC |      NMI |     PREC |      REC |      ARI |
|----|-----------|----------|----------|----------|----------|----------|----------|----------|----------|
|  0 | 102758375 | 0.290102 | 0.763400 | 0.121379 | 0.121450 | 0.716668 | 0.125602 | 0.121450 | 0.634751 |
|  1 | 102758375 | 0.289881 | 0.763200 | 0.073071 | 0.075850 | 0.716801 | 0.073154 | 0.075850 | 0.630876 |
|  2 | 102758375 | 0.290499 | 0.761100 | 0.132062 | 0.133500 | 0.715696 | 0.134442 | 0.133500 | 0.634909 |
|  3 | 102758375 | 0.288695 | 0.764150 | 0.095413 | 0.098150 | 0.717387 | 0.095284 | 0.098150 | 0.634717 |
|  4 | 102758375 | 0.293103 | 0.758750 | 0.124142 | 0.126900 | 0.715118 | 0.127762 | 0.126900 | 0.627984 |
|  5 | 102758375 | 0.289417 | 0.762350 | 0.100434 | 0.099700 | 0.717925 | 0.106166 | 0.099700 | 0.630034 |
|  6 | 102758375 | 0.289903 | 0.763300 | 0.145624 | 0.148700 | 0.716421 | 0.149754 | 0.148700 | 0.632362 |
|  7 | 102758375 | 0.337331 | 0.688300 | 0.098746 | 0.100600 | 0.673209 | 0.105153 | 0.100600 | 0.583970 |
|  8 | 102758375 | 0.290753 | 0.762650 | 0.080585 | 0.080400 | 0.717223 | 0.086493 | 0.080400 | 0.628180 |
|  9 | 102758375 | 0.290330 | 0.762950 | 0.071886 | 0.069750 | 0.716495 | 0.078104 | 0.069750 | 0.634831 |
| 10 | Average   | 0.295001 | 0.755015 | 0.104334 | 0.105500 | 0.712294 | 0.108191 | 0.105500 | 0.627261 |

CPU times: user 56.9 s, sys: 31.1 s, total: 1min 28s
Wall time: 15 s

In [ ]:
perform_evaluate_runs('NusWide', embDim=5, kernelFn='cosine') # no important difference with the default thresold

  0%|          | 0/10 [00:00<?, ?it/s]

---